## 1. 300 - Longest Increasing Subsequence
Given an unsorted array of integers, find the length of longest increasing subsequence.

**idea** for every num, doing a binary search towards tails array  
tails: the smallest number to make array with length i as an increasing subsequence
1. if num > all tails, add it to tails, size += 1
2. if tails[i] < num <= tails[i+1], update tails[i+1]
3. return size

In [1]:
class Solution(object):
    def lengthOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        tails = [0] * len(nums)
        size = 0
        for num in nums:
            l, r = 0, size
            while l < r:
                mid = (l+r)/2
                if tails[mid] < num:
                    l = mid + 1
                else:
                    r = mid
            tails[l] = num
            size = max(l+1, size)
        return size

## 2. 304 - Range Sum Query 2D - Immutable
Given a 2D matrix matrix, find the sum of the elements inside the rectangle defined by its upper left corner (row1, col1) and lower right corner (row2, col2).

**idea** memorize sum of square using the elements as the right-bottom corner.  
square = right-bottom - right-upper - left-bottom + left-upper

In [2]:
class NumMatrix(object):

    def __init__(self, matrix):
        """
        :type matrix: List[List[int]]
        """
        if not matrix: return 
        m, n = len(matrix), len(matrix[0])
        self.mat = [[0 for j in range(n+1)]]
        for i in range(1,m+1):
            self.mat.append([0] + matrix[i-1])
            for j in range(1,n+1):
                self.mat[i][j] = self.mat[i-1][j] + self.mat[i][j-1] - self.mat[i-1][j-1] + self.mat[i][j]   

    def sumRegion(self, row1, col1, row2, col2):
        """
        :type row1: int
        :type col1: int
        :type row2: int
        :type col2: int
        :rtype: int
        """
        return self.mat[row2+1][col2+1] - self.mat[row1][col2+1] - self.mat[row2+1][col1] + self.mat[row1][col1]

# Your NumMatrix object will be instantiated and called as such:
# obj = NumMatrix(matrix)
# param_1 = obj.sumRegion(row1,col1,row2,col2)

## 3. 306 - Additive Number
Additive number is a string whose digits can form additive sequence.  
A valid additive sequence should contain at least three numbers. Except for the first two numbers, each subsequent number in the sequence must be the sum of the preceding two.  
Given a string containing only digits '0'-'9', write a function to determine if it's an additive number.  
Note: Numbers in the additive sequence cannot have leading zeros, so sequence 1, 2, 03 or 1, 02, 3 is invalid.  

### **backtracking**
**idea** backtracking, similar to dfs 

1. update STATUS: path & remain 
2. pass to next step  
3. tell your parents your status  
4. if it's not what you want, return to original status(pop)

In [3]:
class Solution(object):
    def isAdditiveNumber(self, num):
        """
        :type num: str
        :rtype: bool
        """
        def dfs(path, remain):
            if not remain and len(path) > 2:
                return True
            for i in range(len(remain)):
                # num = 0 or num != 0X
                if i == 0 or remain[0] != '0':
                    n = int(remain[:i+1])
                    if len(path) < 2 or path[-2] + path[-1] == n:
                        path.append(n)
                        if dfs(path, remain[i+1:]):
                            return True
                        path.pop()
                    # efficiency 
                    elif path[-2] + path[-1] < n:
                        break
                # i > 0 and remain[0] == '0': don't need continue
                else: break
            return False 
        return dfs([], num)

## 4. 307 - Range Sum Query - Mutable
Given an integer array nums, find the sum of the elements between indices i and j (i ≤ j), inclusive.  
The update(i, val) function modifies nums by updating the element at index i to val.

**idea** binary indexed tree

Use self.c to represent Binary Indexed Tree. Section sums are stored in self.c[1..len(nums)]. **x & -x is lowbit function, which will return x's rightmost bit 1**, e.g. lowbit(7) = 1, lowbit(20) = 4.

self.c[1] = nums[0]

self.c[2] = nums[0] + nums[1]

self.c[3] = nums[2]

self.c[4] = nums[0] + nums[1] + nums[2] + nums[3]

self.c[5] = nums[4]

self.c[6] = nums[4] + nums[5]

self.c[7] = nums[6]

self.c[8] = nums[0] + nums[1] + nums[2] + nums[3] + nums[4] + nums[5] + nums[6] + nums[7]

In [4]:
class NumArray(object):

    def __init__(self, nums):
        """
        :type nums: List[int]
        """
        self.n = len(nums)
        self.lst = nums
        self.t = [0] * (self.n+1)
        for i in range(self.n):
            k = i+1
            while k <= self.n:
                self.t[k] += nums[i]
                k += k & -k

    def update(self, i, val):
        """
        :type i: int
        :type val: int
        :rtype: void
        """
        diff, self.lst[i] = val-self.lst[i], val
        k = i+1
        while k <= self.n:
            self.t[k] += diff
            k += k & -k
        

    def sumRange(self, i, j):
        """
        :type i: int
        :type j: int
        :rtype: int
        """
        ret, j = 0, j+1
        while j > 0:
            ret += self.t[j]
            j -= j & -j
        while i > 0:
            ret -= self.t[i]
            i -= i & -i
        return ret
        

# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(i,val)
# param_2 = obj.sumRange(i,j)

## 5. 309 - Best Time to Buy and Sell Stock with Cooldown
Say you have an array for which the ith element is the price of a given stock on day i.  
Design an algorithm to find the maximum profit. You may complete as many transactions as you like (ie, buy one and sell one share of the stock multiple times) with the following restrictions:  
You may not engage in multiple transactions at the same time (ie, you must sell the stock before you buy again).
After you sell your stock, you cannot buy stock on next day. (ie, cooldown 1 day)

**idea** dp  
use three array: buy,sell,rest to store the biggest revenue when we take this action before day i  

1. buy[i] = max(buy[i-1], rest[i-1]-price)
2. sell[i] = max(sell[i-1], buy[i-1]+price)
3. rest[i] = sell[i-1]

In [5]:
class Solution(object):
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """  
        if not prices or len(prices) == 1: return 0
        buy = [-prices[0], max(-prices[0], -prices[1])]
        sell = [0, max(0,prices[1]-prices[0])]
        for i,price in enumerate(prices):
            if i > 1:
                buy.append(max(buy[i-1],sell[i-2]-price))
                sell.append(max(sell[i-1],buy[i-1]+price))
        return max(buy[-1],sell[-1])